In [1]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

df1 = pd.read_csv('/Volumes/IMVDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['sign', 'glyph'])

start_file = '/Volumes/IMVDrive/cfdb-django/media/start_file.csv'

if 'start_file.csv' in os.listdir('/Volumes/IMVDrive/cfdb-django/media/'):
    os.remove(start_file)

f=open(start_file, 'a+')

df_group = df1.groupby(by=['sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

print("Thresholding number of glyphs per sign at 50...")
first = True
for name,group in df_group:
    if first == True:
        group.reset_index().to_csv(f, header=True)
        first = False;
    elif first == False:
        group.reset_index().to_csv(f, header=None)
print("Thresholding finished.")

/anaconda/envs/labasi/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Thresholding number of glyphs per sign at 50...
Thresholding finished.


In [2]:
from random import shuffle
import glob
import sys
import numpy as np
from PIL import Image
from pathlib import Path

## ------------------------------------------------------------
## FORMAT CUTOM IMAGE DATA SET
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------------------------------------
## I use PIL instead of cv2
## ------------------------------------------------------------

print("FORMAT CUSTOM IMAGE INPUT")

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def createDataRecord(outFileName, addrs, labels):
    print("")
    print('Creating '+outFileName+'...')
    writer = tf.python_io.TFRecordWriter(outFileName)
    for i in range(len(addrs)):
        if not i % 1000:
            print(outFileName+' data: {}/{}'.format(i,len(addrs)))
            sys.stdout.flush()
            
        filename = os.fsdecode('/Volumes/imvDrive/cfdb-django/media/glyph_img/'+addrs.iloc[i])
        if Path(filename).is_file():
            try:
                img = Image.open(filename).convert('L') #convert to grayscale
                img = img.resize((331,331)) 
                label = labels.iloc[i]
                ascii_label = int(''.join(str(ord(c)) for c in label))
                #print("label: "+str(label) + " / "+"ascii: "+str(ascii_label))

                feature_img = {
                    'glyph_img_raw': _bytes_feature(img.tobytes()),#_bytes_feature(tf.compat.as_bytes(img.tostring())),#, # 'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring()))
                    'label_raw': _int64_feature(ascii_label)
                }

                example = tf.train.Example(features=tf.train.Features(feature=feature_img))
                writer.write(example.SerializeToString())
                
            except Exception:
                print("Not written...")
                pass
    writer.close()
    sys.stdout.flush()

FORMAT CUSTOM IMAGE INPUT


In [3]:
glyphtosignpath = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/start_file.csv')
addrs = glyphtosignpath.iloc[:,3]
labels = glyphtosignpath.iloc[:,2]

# divide the data into 80% train, 10% validation, and 10% test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
val_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]

In [4]:
createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)
print("DONE")


Creating train.tfrecords...
train.tfrecords data: 0/3808
Not written...
Not written...
train.tfrecords data: 1000/3808
train.tfrecords data: 2000/3808
Not written...
Not written...
train.tfrecords data: 3000/3808
Not written...
Not written...

Creating val.tfrecords...
val.tfrecords data: 0/476
Not written...
Not written...

Creating test.tfrecords...
test.tfrecords data: 0/477
Not written...
Not written...
Not written...
DONE


In [ ]:
## ------------------------------
## READ INPUT: version 1
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------


import tensorflow as tf
import sys
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

channels = 1
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

def parser(record):
    keys_to_features = {
        "glyph_img_raw": tf.FixedLenFeature([], tf.string),
        "label_raw": tf.FixedLenFeature([], tf.int64)
    }
    
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["glyph_img_raw"], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, shape=[331, 331, channels])
    label = tf.cast(parsed["label_raw"], tf.int32)
    
    return image, label

def input_fn(filenames, train, batch_size=batch_size, buffer_size=2048):
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    dataset = dataset.map(parser)
    
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
        num_repeat = None
    else:
        num_repeat = 1
        
    dataset = dataset.repeat(num_repeat)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    images_batch, labels_batch = iterator.get_next()
    x = {'image': images_batch}
    y = labels_batch

    return x, y

def train_input_fn():
    return input_fn(filenames=['/Volumes/IMVDRIVE/cfdb-django/train.tfrecords'], train=True)

def val_input_fn():
    return input_fn(filenames=["/Volumes/IMVDRIVE/cfdb-django/val.tfrecords"], train=True)

def test_input_fn():
    return input_fn(filenames=["/Volumes/IMVDRIVE/cfdb-django/test.tfrecords"], train=False)
'''
for i in range(5):
    features, labels = train_input_fn()
    img, label = sess.run([features['image'], labels])
    print(label)
'''

for i in range(5):
    try:
        features, labels = train_input_fn()
        img, label = sess.run([features['image'], labels])
        print(img.shape)
    except Exception:
        print("An exception...")
        pass


(100, 331, 331, 1)
(100, 331, 331, 1)
(100, 331, 331, 1)
(100, 331, 331, 1)
(100, 331, 331, 1)


In [ ]:
## ------------------------------
## VARIABLES
## ------------------------------
upload = pd.read_csv('/Volumes/imvDrive/cfdb-django/media/start_file.csv', usecols=['glyph', 'sign']) #
upload = upload.groupby(['sign']).nunique() #
sign_count = len(upload)-1 #
x = tf.placeholder(tf.float32, shape=[batch_size, 331, 331, 1])
l = tf.placeholder(tf.int32, shape=[batch_size])
# y_true_cls = tf.argmax(labels, dimension=1)

## ------------------------------
## CONVOLUTIONAL NEURAL NETWORK
## Adapted from:
## https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
## https://www.tensorflow.org/tutorials/images/deep_cnn
## ------------------------------
# Count number of signs

filter_shape = 5
batches_to_process = 15

def conv_layer1_fn(x,num_input_channels, num_filters):
    W1 = tf.Variable(tf.truncated_normal([filter_shape,filter_shape,num_input_channels,num_filters], dtype=tf.float32))     
    B1 = tf.Variable(tf.truncated_normal([num_filters]))
    conv1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding="SAME") #stides=[batch_stride x_stride y_stride depth_stride]
    biased_conv1 = conv1+B1
    relu_for_conv = tf.nn.relu(biased_conv1)
    pool1 = tf.nn.max_pool(relu_for_conv, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    norm1 = tf.nn.local_response_normalization(pool1,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    return norm1
    
def conv_layer2_fn(layer, num_input_channels, num_filters):
    W2 = tf.Variable(tf.truncated_normal([filter_shape,filter_shape,num_input_channels,num_filters], stddev=0.05)) #https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
    B2 = tf.Variable(tf.truncated_normal([num_filters]))
    conv2 = tf.nn.conv2d(layer, W2, strides=[1, 1, 1, 1], padding="SAME")
    biased_conv2 = conv2+B2
    relu_for_conv = tf.nn.relu(biased_conv2)
    norm2 = tf.nn.local_response_normalization(relu_for_conv,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    return pool2

def flat_layer(layer):                  
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    return layer 

def fc_relu(layer, use_relu, output_num, reshape):
    layer_shape = layer.get_shape()
    W3 = tf.Variable(tf.truncated_normal([layer_shape[1:4].num_elements(), output_num], stddev=0.05))
    ##print("W3: ")
    ##print(W3.shape)
    B3 = tf.Variable(tf.truncated_normal([output_num],stddev=0.03))
    ##print("B3: ")
    ##print(B3.shape)
    std_hypothesis = tf.matmul(layer, W3) + B3
    ##print("hypothesis: ")
    ##print(std_hypothesis)
    if reshape == True:
        std_hypothesis = tf.reshape(std_hypothesis, [-1])
    if use_relu == True:
        fc_relu = tf.nn.relu(std_hypothesis)
        return fc_relu
    else:
        return std_hypothesis
    
conv_layer1 = conv_layer1_fn(x, 1, 32)
conv_layer2 = conv_layer2_fn(conv_layer1, 32, 64)
flat = flat_layer(conv_layer2)
fc_relu1 = fc_relu(flat, use_relu=True, output_num=batch_size, reshape=False)
fc_relu2 = fc_relu(fc_relu1, use_relu=False, output_num=1, reshape=True)
    
## ------------------------------
## OPTIMIZATION SESSION
## Adapted from:
## https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
## ------------------------------
    
y_pred = tf.nn.softmax(fc_relu2)
y_pred_cls = tf.argmax(y_pred, dimension=1, axis=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc_relu2,labels=l)
cost = tf.reduce_mean(cross_entropy)
    
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

## ------------------------------
## CATALYZING PROCESS
## ------------------------------
features, labels = train_input_fn()
batch_counter = 0

sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Run training batches
while features != None or labels != None:
    x_train_batch, label_train_batch = sess.run([features['image'], labels])
    sess.run(optimizer, feed_dict={x: x_train_batch, l: label_train_batch})
    if not i % 10:
        print("training batch "+str(batch_counter)+"...")
    batch_counter = batch_counter+1
    
for i in range(batches_to_process):
    # Process validation batches
    x_valid_batch, label_valid_batch = val_input_fn() #data.valid.next_batch(train_batch_size)
    feed_dict_val = {x: x_valid_batch.eval(), labels: label_train_batch.eval()}
    val_loss = sess.run(cost, feed_dict=feed_dict_val)
    print("validation batches")   


Instructions for updating:
Use the `axis` argument instead


In [ ]:
## ------------------------------
## PREDICTION
## ------------------------------
y_pred = tf.nn.softmax(fc_relu2)
y_pred_cls = tf.argmax(y_pred, dimension=1)
    